In [1]:
from gatv2 import GATV2

import torch
from torchdrug import datasets

dataset = datasets.ClinTox("~/molecule-datasets/")
# dataset = datasets.ZINC250k("~/molecule-datasets/", kekulize=True, atom_feature="symbol")

Constructing molecules from SMILES:   0%|          | 0/1484 [00:00<?, ?it/s]/Users/ca/Desktop/graph-generative-models/torchdg0.2.0/torchdrug/data/feature.py:42: UserWarning: Unknown value `*`
  warnings.warn("Unknown value `%s`" % x)
[02:16:55] Explicit valence for atom # 0 N, 5, is greater than permitted
/Users/ca/Desktop/graph-generative-models/torchdg0.2.0/torchdrug/data/feature.py:42: UserWarning: Unknown value `Tc`
  warnings.warn("Unknown value `%s`" % x)
/Users/ca/Desktop/graph-generative-models/torchdg0.2.0/torchdrug/data/feature.py:42: UserWarning: Unknown value `Fe`
  warnings.warn("Unknown value `%s`" % x)
/Users/ca/Desktop/graph-generative-models/torchdg0.2.0/torchdrug/data/feature.py:42: UserWarning: Unknown value `Al`
  warnings.warn("Unknown value `%s`" % x)
Constructing molecules from SMILES:   6%|▌         | 83/1484 [00:00<00:01, 816.62it/s]/Users/ca/Desktop/graph-generative-models/torchdg0.2.0/torchdrug/data/feature.py:42: UserWarning: Unknown value `Ca`
  warnings.wa

In [ ]:
models.GearNet(input_dim=dataset.node_feature_dim,
                    num_relation=dataset.num_bond_type,
                    hidden_dims=[64, 64, 64], batch_norm=True)

In [ ]:
from torch_geometric.utils import degree
from pna import PNA

In [ ]:
deg = torch.zeros(10, dtype=torch.long)
for data in dataset:
    graph = data['graph']
    d = degree(graph.edge_list[:, 1], num_nodes=graph.num_node, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

PNA(input_dim=dataset.node_feature_dim,
                  hidden_dim=64, num_layer=3,
                  edge_input_dim=dataset.edge_feature_dim,
                  num_relation=dataset.num_bond_type,
                  aggregators=['mean'],
                  scalers=['identity'],
                  deg=deg, batch_norm=True)

In [ ]:
from gsn import GSN
from gsn import prepare_GSN_dataset
prepare_GSN_dataset(dataset, max_cycle=8)

GSN(input_dim=dataset.node_feature_dim,
                  hidden_dim=64, num_layer=3,
                  edge_input_dim=dataset.edge_feature_dim,
                  num_relation=dataset.num_bond_type,
                  batch_norm=True)

In [2]:
from torchdrug import core, tasks

model = GATV2(input_dim=dataset.node_feature_dim,
                    num_relation=dataset.num_bond_type,
                    hidden_dims=[256, 256, 256, 256], batch_norm=True)
task = tasks.GCPNGeneration(model, dataset.atom_types, max_edge_unroll=12,
                            max_node=38, criterion="nll")

In [3]:
from torch import nn, optim
optimizer = optim.Adam(task.parameters(), lr = 1e-3)
solver = core.Engine(task, dataset, None, None, optimizer,
                     batch_size=128, log_interval=10)

solver.train(num_epoch=1)

02:16:59   Preprocess training set
02:16:59   {'batch_size': 128,
 'class': 'core.Engine',
 'gpus': None,
 'gradient_interval': 1,
 'log_interval': 10,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'class': 'optim.Adam',
               'eps': 1e-08,
               'lr': 0.001,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'agent_update_interval': 10,
          'atom_types': [0, 1, 5, ..., 80, 81, 83],
          'baseline_momentum': 0.9,
          'class': 'tasks.GCPNGeneration',
          'criterion': 'nll',
          'gamma': 0.9,
          'hidden_dim_mlp': 128,
          'max_edge_unroll': 12,
          'max_node': 38,
          'model': {'activation': 'relu',
                    'batch_norm': True,
                    'class': 'GATV2',
                    'concat_hidden': False,
                    'edge_input_dim': None,
                    'hidden_dims': [256, 256, 256, 256],
      

KeyboardInterrupt: 